<h1>Web scraping  Higg MSI Index</h1>

<h3>Import necessary modules</h3>

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import requests

In [2]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time

In [3]:
keyword = '1'
url = 'https://msi.higg.org/process/' + keyword

In [16]:
def get_indexdata(url, keyword):
    
    # create a new Firefox session
    driver = webdriver.Chrome('/Users/Jeremy/Documents/Columbia MS&E/Summer Startup Track 2020/Higg-MSI/chromedriver')
    driver.implicitly_wait(30)
    driver.get(url+keyword)

    #Selenium hands the page source to Beautiful Soup
    result_page = BeautifulSoup(driver.page_source, 'lxml')
    
    #scraping data
    try:

        table_title = ['Impact area','MSI Score']

        meta_information = result_page.find(id='meta_information').h2.text

        meta_data_list=[]
        for row in result_page.find(id='meta_information').find_all("li"):
            meta_data_list.append(row)

        start = meta_data_list[0].text.find('Primary Source: ') + len("Primary Source: ")

        primary_source = meta_data_list[0].text[start:]

        start = meta_data_list[1].text.find('Last Publication or Review date: ') + len("Last Publication or Review date: ")

        publication_date = meta_data_list[1].text[start:]

        start = meta_data_list[2].text.find('Geography: ') + len("Geography: ")

        geography = meta_data_list[2].text[start:]

        start = meta_data_list[3].text.find('Data Generator: ') + len("Data Generator: ")

        data_generator = meta_data_list[3].text[start:]

        title = result_page.find_all('div', class_="header-title")[0].h1.text

        description = result_page.find_all('div', class_ = 'paragraphs-text')[0].text.strip()
    except:
        pass
    
    try:
        element_list = []

        for row in result_page.find_all('table', class_='sac-process-scores impacts'):
            for element in row.tbody.find_all('tr'):
                element_list.append(element)

        start = element_list[0].text.strip().find('Global Warming\n                \n\n                    ') + len('Global Warming\n                \n\n                    ')

        global_warming = element_list[0].text.strip()[start:]

        start = element_list[1].text.strip().find('Eutrophication\n                \n\n                    ') + len('Eutrophication\n                \n\n                    ')

        eutrophication = element_list[1].text.strip()[start:]

        start = element_list[2].text.strip().find('Water Scarcity\n                \n\n                    ') + len('Water Scarcity\n                \n\n                    ')

        water_scarcity = element_list[2].text.strip()[start:]

        start = element_list[3].text.strip().find('Abiotic Resource Depletion, Fossil Fuels\n                \n\n                    ') + len('Abiotic Resource Depletion, Fossil Fuels\n                \n\n                    ')

        abiotic_rd = element_list[3].text.strip()[start:]
        
        impact_title = ['Global Warming', 'Eutrophication', 'Water Scarcity', 'Abiotic Resource Depletion']
        impact_values = [global_warming, eutrophication, water_scarcity, abiotic_rd]
        
    except:
        impact_title = ['Global Warming', 'Eutrophication', 'Water Scarcity', 'Abiotic Resource Depletion']
        impact_values = [None, None, None, None]
        pass
    
    try:
        overall_impact = result_page.find_all('div', class_ = 'chart-container chart__large')[0].strong.text.strip()
        overall_impact_list = [overall_impact for i in range(len(impact_values))]
    except:
        overall_impact_list = [None for i in range(len(impact_values))] 
        pass
    
    try:
        last_table = str(result_page.find(id='sac_data_quality_indicators'))

        data_quality = pd.read_html(last_table, header=0)[0]

        data_quality.columns = ['Quality Category', 'Value', 'DQ_Description', 'Data Quality']

        data_quality = data_quality.iloc[0:-1,:]
        
    except:
        none = [None for i in range(len(impact_values))]
        data_quality = pd.DataFrame([none, none, none, none], columns=['Quality Category', 'Value', 'DQ_Description', 'Data Quality'] )
        pass

    try:
        
    
        start = result_page.find(id='modeling_notes').div.div.text.strip().find('Modeling Notes\n\n            ') + len('Modeling Notes\n\n            ')
        modeling_notes = result_page.find(id='modeling_notes').div.div.text.strip()[start:]

        title_list = [title for i in range(len(impact_values)) ]
        description_list = [description for i in range(len(impact_values)) ]
        modeling_notes_list = [modeling_notes for i in range(len(impact_values)) ]


        primary_source_list = [primary_source for i in range(len(impact_values))]
        publication_date_list = [publication_date for i in range(len(impact_values))]
        geography_list = [geography for i in range(len(impact_values))]
        data_generator_list = [data_generator for i in range(len(impact_values))]

        applicable_mat_list=[]
    
    except:
        pass
    
    try:
        for row in result_page.find(id='materials').tbody.find_all('tr'):
            applicable_mat_list.append(row)
        applicable_materials = []
        for i in range(len(applicable_mat_list)):
            temp = applicable_mat_list[i].find('strong', class_= 'row-anchor').text
            applicable_materials.append(temp)

    except:
        applicable_materials = [None for i in range(len(impact_values))]
        pass
    
    driver.quit()
    
    
    try:
        overall_score = float(global_warming)+ float(eutrophication)+ float(water_scarcity)+ float(abiotic_rd)
        df = pd.DataFrame([title, float(global_warming), float(eutrophication), float(water_scarcity), float(abiotic_rd), overall_score, applicable_materials]).transpose()
        
        df.columns = ['Process', 'GlobalWarming', 'Eutrophication', 'WaterScarcity', 'AbioticResourceDepletion', 'Overall', 'ApplicableMaterials']
    
    except:
        df = pd.DataFrame()

    return df


In [17]:
keyword = '1'
url = 'https://msi.higg.org/process/' 
get_indexdata(url,keyword)

,Process,GlobalWarming,Eutrophication,WaterScarcity,AbioticResourceDepletion,Overall,ApplicableMaterials
0,"Leather finishing, 1 drum operation (dyeing, w...",0.4,0.2,0.059,0.6,1.259,"[Cow leather, Goat leather, Kangaroo leather, ..."


In [18]:
#aggregating all data together
final_df = pd.DataFrame()
for i in range(503):
    keyword = str(i+1)
    table = get_indexdata(url = 'https://msi.higg.org/process/', keyword = keyword)
    final_df = pd.concat([final_df,table],axis=0)
    


In [19]:
final_df

,Process,GlobalWarming,Eutrophication,WaterScarcity,AbioticResourceDepletion,Overall,ApplicableMaterials
0,"Leather finishing, 1 drum operation (dyeing, w...",0.4,0.2,0.059,0.6,1.259,"[Cow leather, Goat leather, Kangaroo leather, ..."
0,"Leather finishing, 2 drum operations (dyeing, ...",0.8,0.3,0.1,1.2,2.4,"[None, None, None, None]"
0,"Leather finishing, 3 drum operations (dyeing, ...",1.3,0.5,0.2,1.8,3.8,"[None, None, None, None]"
0,"Synthetic leather substrate, 70% Polyester/30%...",4.1,2.5,0.7,5.4,12.7,[Polyurethane (PU) synthetic leather]
0,"Acetylated and regenerated cellulose, from pin...",7.6,5.5,4.1,6.8,24,"[Acetate, Triacetate fabric]"
0,"Acrylonitrile butadiene styrene (ABS) plastic,...",5.3,1.6,1.1,8.2,16.2,[Acrylonitrile butadiene styrene (ABS) plastic]
0,"Acrylonitrile, fossil fuel based",3.3,7.8,0.9,6.5,18.5,[Acrylic fabric]
0,"Tanning, aldehyde process",4.5,2,0.6,5.1,12.2,"[None, None, None, None]"
0,"Aluminum, primary/virgin",21.2,8.9,1.1,14.3,45.5,[Aluminum]
0,Antimicrobial/antifungal/antibacterial,0,0,0,0,0,"[None, None, None, None]"


In [21]:
final_df.to_csv("process-score.csv")